<a href="https://colab.research.google.com/github/facahto/Kaggle-competition-practice/blob/main/forecasting-store_sales/Stores_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd, numpy as np
import torch
import sqlite3
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

Data

In [10]:
# Holiday Event
h_e = pd.read_csv('holidays_events.csv')
display(h_e.head(3))
print()
print(h_e.info())
#display(h_e[h_e.duplicated(keep=False)]) ----- NO DUPLICATED DATA

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB
None


In [11]:
# Oil
oil = pd.read_csv('oil.csv')
oil['dcoilwtico'].fillna(float(oil['dcoilwtico'].mean()), inplace=True)
display(oil.head(3))
print()
print(oil.info())
#display(oil[oil.duplicated(keep=False)])  ----- NO DUPLICATED DATA

,date,dcoilwtico
0,2013-01-01,67.714366
1,2013-01-02,93.140000
2,2013-01-03,92.970000



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB
None


In [12]:
# Stores
stores = pd.read_csv('stores.csv')
display(stores.head(3))
print()
print(stores.info())
#display(stores[stores.duplicated(keep=False)])  ----- NO DUPLICATED DATA

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB
None


In [13]:
# Transactions
ts = pd.read_csv('transactions.csv')
display(ts.head(3))
print()
print(ts.info())
#display(ts[ts.duplicated(keep=False)])  ----- NO DUPLICATED DATA

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB
None


In [36]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
display(train.head())

IntCastingNaNError: ignored

In [39]:
# Preprocessing train data
store_nan     = train.isna().sum()
store_nan_row = train[train.isna().any(axis=1)]
print("Jumlah nilai NaN dalam kolom 'store_nbr':", store_nan)
print(store_nan_row)

train['family']= label_encoder.fit_transform(train['family'])
test['family']= label_encoder.fit_transform(test['family'])

Jumlah nilai NaN dalam kolom 'store_nbr': id             0
date           0
store_nbr      1
family         0
sales          1
onpromotion    1
dtype: int64
              id date  store_nbr  family  sales  onpromotion
1827558  1827558   20        NaN      33    NaN          NaN


In [16]:
sample = pd.read_csv('sample_submission.csv')
display(sample)

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [34]:
data = {'Kolom1': ['10', '20', '30'],
        'Kolom2': ['40', '50', '60']}

df = pd.DataFrame(data)

# Tambahkan kolom baru yang berisi penjumlahan dua kolom
df['Kolom3'] = df['Kolom1'] + df['Kolom2']

# Konversi kembali Kolom3 ke string jika diperlukan
df['Kolom3'] = df['Kolom3'].astype(str)

# Tampilkan DataFrame
print(df)

  Kolom1 Kolom2 Kolom3
0     10     40   1040
1     20     50   2050
2     30     60   3060
